Creo un file csv, il quale per ogni specie di funghi che trova nel Dataset mi inserisce, dove possibile:
    - edibilità: velenoso, edibile, non determinata
    - caratteristiche
    - Habitat

In [4]:
import os
import wikipediaapi
import csv
import time

# Inizializzazione dell'API di Wikipedia con un User-Agent personalizzato
user_agent = "MyFungiBot/1.0 (davidigno11@gmail.com)"
headers = {"User-Agent": user_agent}

# Creiamo l'oggetto Wikipedia passando i headers con User-Agent
wiki = wikipediaapi.Wikipedia('en', headers=headers)

# Funzione per ottenere informazioni sulla tossicità, caratteristiche e habitat di una specie fungina
def get_fungi_info(species_name, timeout=60):
    try:
        page = wiki.page(species_name)
        
        if not page.exists():
            return "Information not found", "Information not found"
        
        text = page.text.lower()
        
        # Determinazione della tossicità
        if "poisonous" in text or "toxic" in text:
            toxicity = "Poisonous"
        elif "inedible" in text:
            toxicity = "Inedible"
        elif "edible" in text:
            toxicity = "Edible"
        else:
            toxicity = "Information unclear"

        habitats = []
        habitat_str = ""
        if toxicity != "Information unclear":
            # Estrazione dell'habitat
            if "forest" in text:
                habitats.append("Forest")
            if "grassland" in text:
                habitats.append("Grassland")
            if "deciduous" in text:
                habitats.append("Deciduous trees")
            if "coniferous" in text:
                habitats.append("Coniferous trees")
            if "soil" in text:
                habitats.append("Soil")
            if "logs" in text or "wood" in text:
                habitats.append("Logs/Wood")
            
            habitat_str = ", ".join(habitats) if habitats else "Habitat unclear"
        
        return toxicity, habitat_str
    except Exception as e:
        return f"Error: {str(e)}", "Error"

# Funzione per ottenere caratteristiche dettagliate del fungo da Wikipedia
def get_fungi_details(species_name):
    try:
        page = wiki.page(species_name)
        if not page.exists():
            return "No detailed characteristics found"
        
        # Restituiamo direttamente i primi 500 caratteri (puoi regolare questo valore)
        return page.summary[:500]
    except Exception as e:
        return f"Error: {str(e)}"

# Percorso della cartella delle specie (adatta questo percorso al tuo dataset)
image_dir = 'MIND.Funga_Dataset'

# Elenco delle specie (estratto dai nomi delle cartelle nel dataset)
species_list = [folder for folder in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, folder))]

# Percorso del file CSV originale e del nuovo file aggiornato
output_file = 'fungi_info.csv'

# Creazione del CSV con nome scientifico, tossicità, caratteristiche, habitat e dettagli
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Species', 'Toxicity', 'Characteristics', 'Habitat'])  # Intestazione del CSV
    
    # Per ogni specie nel dataset, cerchiamo le informazioni
    for species in species_list:
        species_name = species.replace('_', ' ')  # Rimuoviamo eventuali underscore nei nomi delle cartelle
        
        # Raccogli le informazioni generali dalla funzione get_fungi_info
        toxicity, habitat = get_fungi_info(species_name)
        
        if toxicity != "Information unclear" and toxicity != "Information not found":
            # Raccogli le informazioni dettagliate dalla funzione get_fungi_details
            characteristics = get_fungi_details(species_name)
            
            # Scrivi le informazioni nel CSV
            writer.writerow([species, toxicity, characteristics, habitat])
        
        # Pausa per evitare sovraccarico di richieste
        time.sleep(3)  # Pausa di 3 secondi

print(f"CSV created successfully: {output_file}")


CSV created successfully: fungi_info.csv


Filtro il Dataset di partenza per tenere esclusivamente le immagini delle specie di funghi dei quali sappiamo edibilità, caratteristiche e habitat

In [16]:
import os
import shutil
import csv
import re

# Percorsi delle cartelle
original_dataset_dir = 'Dataset/MIND.Funga App images dataset from Neotropical macrofungi used to train an artificial neural network to recognize fungal species'
output_dataset_dir = 'Filtered_Fungi_Dataset'

# Percorso del file CSV aggiornato
csv_file = 'fungi_info_detailed.csv'

# Funzione per normalizzare i nomi dei file (rimuovere caratteri speciali)
def normalize_filename(filename):
    return re.sub(r'[^a-zA-Z0-9._-]', '_', filename)  # Sostituisce i caratteri non alfanumerici con underscore

# Funzione per creare il nuovo dataset
def create_filtered_dataset(original_dir, output_dir, csv_path):
    # Creazione della cartella di output se non esiste
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Leggi il file CSV per ottenere i nomi delle specie edibili, velenose, o "edibile?"
    with open(csv_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        species_list = [row['Species'] for row in reader if row['Toxicity'] in ['Not poisonous', 'Poisonous', 'Edible?']]  # Filtro per tossicità

    # Copia le immagini delle specie corrispondenti
    for species in species_list:
        source_folder = os.path.join(original_dir, species)
        target_folder = os.path.join(output_dir, species)

        if os.path.exists(source_folder):
            # Se la cartella di destinazione non esiste, creiamola
            if not os.path.exists(target_folder):
                os.makedirs(target_folder)

            # Scorri tutti i file nella cartella della specie
            for filename in os.listdir(source_folder):
                source_file = os.path.join(source_folder, filename)
                target_file = os.path.join(target_folder, normalize_filename(filename))
                
                try:
                    # Normalizza i percorsi
                    source_file = os.path.normpath(source_file)
                    target_file = os.path.normpath(target_file)

                    # Copia solo i file immagine
                    if os.path.isfile(source_file):
                        shutil.copy(source_file, target_file)
                        print(f"Copied {filename} to {species}")
                except Exception as e:
                    print(f"Error copying {filename}: {e}")
        else:
            print(f"Warning: Folder for species {species} not found in the original dataset.")

# Esegui la funzione per creare il nuovo dataset
create_filtered_dataset(original_dataset_dir, output_dataset_dir, csv_file)

print(f"Nuovo dataset creato in: {output_dataset_dir}")


Copied VP057 P4 MG (11).jpg to Amanita muscaria
Copied VP057 P4 MG (3).jpg to Amanita muscaria
Copied VP057 P4 MG (4).jpg to Amanita muscaria
Copied VP057 P4 MG (5).jpg to Amanita muscaria
Copied VP057 P4 MG (8).jpg to Amanita muscaria
Copied VP069 P4 MG (2).jpg to Amanita muscaria
Copied VP069 P4 MG (5).jpg to Amanita muscaria
Copied VP074 P4 MG (1).jpg to Amanita muscaria
Copied VP074 P4 MG (2).jpg to Amanita muscaria
Copied VP074 P4 MG (3).jpg to Amanita muscaria
Copied VP074 P4 MG (6).jpg to Amanita muscaria
Copied FK nc (2).jpg to Coprinellus disseminatus
Copied FK nn (1).jpg to Coprinellus disseminatus
Copied FK nn (2).jpg to Coprinellus disseminatus
Copied FK nn (3).jpg to Coprinellus disseminatus
Copied 00-IMG_3772.jpg to Cordyceps militaris
Copied IMG_3760.jpg to Cordyceps militaris
Copied IMG_3761.jpg to Cordyceps militaris
Copied IMG_3763.jpg to Cordyceps militaris
Copied IMG_3771.jpg to Cordyceps militaris
Copied IMG_3773.jpg to Cordyceps militaris
Copied IMG_3980.jpg to Co

Codice per calcolare dimensione minima e media delle immagini del dataset 

In [4]:
import os
from PIL import Image
import numpy as np

# Percorso della directory generale
DATASET_DIR = "MIND.Funga_Dataset"

def count_small_images(dataset_dir):
    image_sizes = []  # Lista per memorizzare le dimensioni delle immagini (larghezza, altezza)
    species_count = 0
    image_count = 0

    # Itera attraverso tutte le sottocartelle (specie) e le immagini
    for species_folder in os.listdir(dataset_dir):
        species_path = os.path.join(dataset_dir, species_folder)
        if os.path.isdir(species_path):  # Controlla se è una cartella
            species_count += 1
            for image_file in os.listdir(species_path):
                image_path = os.path.join(species_path, image_file)
                try:
                    # Apri l'immagine e ottieni le dimensioni
                    with Image.open(image_path) as img:
                        image_sizes.append(img.size)  # Aggiungi (larghezza, altezza) alla lista
                        image_count += 1
                except Exception as e:
                    print(f"Errore con l'immagine: {image_path}. Dettagli: {e}")

    # Analisi delle dimensioni
    if image_sizes:
        # Converti la lista in un array NumPy per analisi
        sizes_array = np.array(image_sizes)  # Array di [larghezza, altezza]
        mean_width, mean_height = sizes_array.mean(axis=0)  # Dimensioni medie
        threshold_width = mean_width - 100
        threshold_height = mean_height - 100

        # Conta quante immagini sono sotto la soglia
        small_images = [size for size in sizes_array if size[0] < threshold_width or size[1] < threshold_height]

        print(f"Specie totali analizzate: {species_count}")
        print(f"Immagini totali analizzate: {image_count}")
        print(f"Dimensioni medie: {mean_width:.2f}x{mean_height:.2f}")
        print(f"Soglia per immagini molto piccole: {threshold_width:.2f}x{threshold_height:.2f}")
        print(f"Immagini con dimensioni molto piccole: {len(small_images)}")
    else:
        print("Nessuna immagine trovata nel dataset.")

# Esegui l'analisi
count_small_images(DATASET_DIR)


Specie totali analizzate: 509
Immagini totali analizzate: 18387
Dimensioni medie: 996.51x996.28
Soglia per immagini molto piccole: 896.51x896.28
Immagini con dimensioni molto piccole: 86


In [6]:
import os
import shutil
import stat

# Percorso della directory generale
DATASET_DIR = "MIND.Funga_Dataset"

def handle_remove_readonly(func, path, exc_info):
    """
    Funzione callback per rimuovere il flag di sola lettura.
    """
    os.chmod(path, stat.S_IWRITE)  # Rimuove il flag di sola lettura
    func(path)

def find_and_remove_folders_with_few_images(dataset_dir, threshold=10):
    folders_with_few_images = []  # Lista per memorizzare le cartelle con meno di 'threshold' immagini
    
    for species_folder in os.listdir(dataset_dir):
        species_path = os.path.join(dataset_dir, species_folder)
        
        if os.path.isdir(species_path):  # Controlla se è una cartella
            # Conta il numero di immagini nella cartella
            image_count = len([f for f in os.listdir(species_path) if os.path.isfile(os.path.join(species_path, f))])
            
            # Verifica se il numero di immagini è inferiore alla soglia
            if image_count < threshold:
                folders_with_few_images.append((species_folder, image_count))
                try:
                    # Rimuove la cartella forzando i permessi
                    shutil.rmtree(species_path, onerror=handle_remove_readonly)
                    print(f"Rimossa la cartella: {species_folder} con {image_count} immagini")
                except Exception as e:
                    print(f"Errore durante l'eliminazione di {species_folder}: {e}")
    
    # Stampa il riepilogo
    print(f"Cartelle rimosse con meno di {threshold} immagini: {len(folders_with_few_images)}")
    for folder_name, count in folders_with_few_images:
        print(f"- {folder_name}: {count} immagini")

# Esegui la funzione
find_and_remove_folders_with_few_images(DATASET_DIR, threshold=10)


Rimossa la cartella: Calvatia guzmanii con 8 immagini
Rimossa la cartella: Campanella cf. aeruginea con 8 immagini
Rimossa la cartella: Cheilymenia sp 1 con 9 immagini
Rimossa la cartella: Chlorophyllum sp con 8 immagini
Rimossa la cartella: Clavaria neofossicola con 9 immagini
Rimossa la cartella: Coccocarpia sp con 9 immagini
Rimossa la cartella: Coenogonium sp con 8 immagini
Rimossa la cartella: Coltricia sp con 9 immagini
Rimossa la cartella: Crocodia aurata con 9 immagini
Rimossa la cartella: Dentipellicula cf. leptodon con 9 immagini
Rimossa la cartella: Dicephalospora rufocornea con 8 immagini
Rimossa la cartella: Diplomitoporus venezuelicus con 0 immagini
Rimossa la cartella: Eichleriella cf. alliciens con 9 immagini
Rimossa la cartella: Entoloma dysthales f acystidiatum con 7 immagini
Rimossa la cartella: Entoloma sp con 8 immagini
Rimossa la cartella: Erioscyphella sp con 7 immagini
Rimossa la cartella: Fuscoporia sp con 8 immagini
Rimossa la cartella: Heterodermia sp con 8 i